# Laboratorio 2 - Detección de Phishing
## CC3045 Inteligencia Artificial

**Grupo:**
- Nombre 1
- Nombre 2
- Nombre 3

---
## TASK 2 - Preparación de Datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Para que las gráficas se vean bien
plt.style.use('default')
%matplotlib inline


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Emadlg\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Emadlg\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Emadlg\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

### Cargar el dataset

In [2]:
# Cargar datos - ajusta el nombre del archivo según lo descargues de Kaggle
df = pd.read_csv('dataset_phishing.csv')

print(f"Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
df.head()

Datos cargados: 11430 filas, 89 columnas


,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [ ]:
# Ver info general
print("\nInformación del dataset:")
df.info()

# Revisar si hay nulos
print(f"\nValores nulos: {df.isnull().sum().sum()}")

### Limpiar datos

In [ ]:
# Eliminar nulos si hay
df = df.dropna()

# Eliminar columnas que no sirven (IDs, URLs, etc.)
# Ajusta esto según tu dataset
columnas_eliminar = []  # Por ejemplo: ['url', 'id']
if columnas_eliminar:
    df = df.drop(columns=columnas_eliminar)

print(f"Dataset limpio: {df.shape}")

### Convertir variables categóricas a números

In [ ]:
# Convertir texto a números
from sklearn.preprocessing import LabelEncoder

for col in df.columns:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        print(f"Convertida: {col}")

### Separar features y target

In [ ]:
# Separar X (features) y y (target)
# Asume que 'status' es la columna objetivo - ajusta si es diferente
target_col = 'status'

X = df.drop(target_col, axis=1)
y = df[target_col]

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nDistribución de clases:")
print(y.value_counts())

### Seleccionar las 2 mejores features (para visualizar en 2D)

In [ ]:
# Calcular correlación de cada feature con el target
correlaciones = X.corrwith(y).abs().sort_values(ascending=False)

print("Correlaciones con el target:")
print(correlaciones)

# Tomar las 2 con mayor correlación
top_features = correlaciones.head(2).index.tolist()
print(f"\n2 mejores features: {top_features}")

X_2d = X[top_features].values

In [ ]:
# Ver cómo se distribuyen los datos
plt.figure(figsize=(10, 6))
plt.scatter(X_2d[y==0, 0], X_2d[y==0, 1], c='blue', label='Legítimo', alpha=0.5)
plt.scatter(X_2d[y==1, 0], X_2d[y==1, 1], c='red', label='Phishing', alpha=0.5)
plt.xlabel(top_features[0])
plt.ylabel(top_features[1])
plt.title('Datos antes de escalar')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Escalar los datos (MUY IMPORTANTE)

In [ ]:
# StandardScaler: convierte los datos a media=0 y std=1
# Fórmula: (x - media) / desviación_estándar

scaler = StandardScaler()
X_2d_scaled = scaler.fit_transform(X_2d)

print("Datos escalados")
print(f"Media: {X_2d_scaled.mean(axis=0)}")
print(f"Std: {X_2d_scaled.std(axis=0)}")

In [ ]:
# Ver la diferencia después de escalar
plt.figure(figsize=(10, 6))
plt.scatter(X_2d_scaled[y==0, 0], X_2d_scaled[y==0, 1], c='blue', label='Legítimo', alpha=0.5)
plt.scatter(X_2d_scaled[y==1, 0], X_2d_scaled[y==1, 1], c='red', label='Phishing', alpha=0.5)
plt.xlabel(f'{top_features[0]} (escalado)')
plt.ylabel(f'{top_features[1]} (escalado)')
plt.title('Datos después de escalar')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Split: 80% train, 20% test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_2d_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")

---
## TASK 3 - Implementación Manual

### Regresión Logística desde cero

In [3]:
class RegresionLogistica:
    def __init__(self, learning_rate=0.1, epochs=1000):
        self.lr = learning_rate
        self.epochs = epochs
        self.w = None
        self.b = None
        self.losses = []
    
    def sigmoid(self, z):
        # σ(z) = 1 / (1 + e^-z)
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def fit(self, X, y):
        m, n = X.shape
        self.w = np.zeros(n)
        self.b = 0
        
        for epoch in range(self.epochs):
            # Forward: calcular predicciones
            z = np.dot(X, self.w) + self.b
            y_pred = self.sigmoid(z)
            
            # Calcular loss (Binary Cross-Entropy)
            epsilon = 1e-15
            y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
            loss = -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
            self.losses.append(loss)
            
            # Calcular gradientes
            dw = (1/m) * np.dot(X.T, (y_pred - y))
            db = (1/m) * np.sum(y_pred - y)
            
            # Actualizar pesos
            self.w -= self.lr * dw
            self.b -= self.lr * db
            
            if (epoch + 1) % 100 == 0:
                print(f"Época {epoch+1}/{self.epochs}, Loss: {loss:.4f}")
    
    def predict_proba(self, X):
        z = np.dot(X, self.w) + self.b
        return self.sigmoid(z)
    
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

In [4]:
# Entrenar
lr_manual = RegresionLogistica(learning_rate=0.1, epochs=1000)
lr_manual.fit(X_train, y_train.values)

NameError: name 'X_train' is not defined

### Gráfica de costo

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(lr_manual.losses)
plt.xlabel('Época')
plt.ylabel('Loss (Binary Cross-Entropy)')
plt.title('Evolución del costo durante entrenamiento')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Loss inicial: {lr_manual.losses[0]:.4f}")
print(f"Loss final: {lr_manual.losses[-1]:.4f}")

### Frontera de decisión

In [ ]:
def plot_decision_boundary(model, X, y, title):
    # Crear una malla de puntos
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Predecir para cada punto
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Graficar
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, levels=20, cmap='RdYlBu', alpha=0.6)
    plt.colorbar(label='P(Phishing)')
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2, linestyles='--')
    
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Legítimo', edgecolors='k', s=50)
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Phishing', edgecolors='k', s=50)
    
    plt.xlabel(top_features[0])
    plt.ylabel(top_features[1])
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_decision_boundary(lr_manual, X_train, y_train.values, 
                      'Frontera de Decisión - Regresión Logística')

### KNN desde cero

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        predictions = []
        for x in X:
            # Calcular distancia euclidiana a todos los puntos de entrenamiento
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            
            # Encontrar los k vecinos más cercanos
            k_indices = np.argsort(distances)[:self.k]
            k_labels = self.y_train[k_indices]
            
            # Votación mayoritaria
            prediction = np.bincount(k_labels).argmax()
            predictions.append(prediction)
        
        return np.array(predictions)

In [ ]:
# Entrenar KNN
knn_manual = KNN(k=3)
knn_manual.fit(X_train, y_train.values)
print("KNN entrenado")

### Frontera de decisión KNN

In [ ]:
def plot_knn_boundary(knn, X_train, y_train, X_test, y_test, title):
    h = 0.02
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    print("Generando mapa de decisión (puede tardar un poco)...")
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='RdYlBu', levels=1)
    
    plt.scatter(X_train[y_train==0, 0], X_train[y_train==0, 1], 
                c='blue', label='Legítimo (train)', alpha=0.6, s=50)
    plt.scatter(X_train[y_train==1, 0], X_train[y_train==1, 1], 
                c='red', label='Phishing (train)', alpha=0.6, s=50)
    
    plt.scatter(X_test[y_test==0, 0], X_test[y_test==0, 1], 
                c='blue', marker='s', label='Legítimo (test)', s=100, edgecolors='black', linewidths=2)
    plt.scatter(X_test[y_test==1, 0], X_test[y_test==1, 1], 
                c='red', marker='s', label='Phishing (test)', s=100, edgecolors='black', linewidths=2)
    
    plt.xlabel(top_features[0])
    plt.ylabel(top_features[1])
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_knn_boundary(knn_manual, X_train, y_train.values, X_test, y_test.values,
                 'Frontera de Decisión - KNN (K=3)')

---
## TASK 4 - Comparación con sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Entrenar modelos de sklearn
lr_sklearn = LogisticRegression(max_iter=1000)
lr_sklearn.fit(X_train, y_train)

knn_sklearn = KNeighborsClassifier(n_neighbors=3)
knn_sklearn.fit(X_train, y_train)

print("Modelos de sklearn entrenados")

In [ ]:
# Hacer predicciones
y_pred_lr_manual = lr_manual.predict(X_test)
y_pred_lr_sklearn = lr_sklearn.predict(X_test)
y_pred_knn_manual = knn_manual.predict(X_test)
y_pred_knn_sklearn = knn_sklearn.predict(X_test)

In [ ]:
# Calcular métricas
def mostrar_metricas(y_true, y_pred, nombre):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    
    print(f"\n{'='*50}")
    print(f"{nombre}")
    print(f"{'='*50}")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"\nMatriz de Confusión:")
    print(confusion_matrix(y_true, y_pred))
    
    return acc, prec, rec

# Comparar todos
r1 = mostrar_metricas(y_test, y_pred_lr_manual, "Regresión Logística Manual")
r2 = mostrar_metricas(y_test, y_pred_lr_sklearn, "Regresión Logística sklearn")
r3 = mostrar_metricas(y_test, y_pred_knn_manual, "KNN Manual (K=3)")
r4 = mostrar_metricas(y_test, y_pred_knn_sklearn, "KNN sklearn (K=3)")

In [ ]:
# Tabla comparativa
resultados = pd.DataFrame({
    'LR Manual': r1,
    'LR sklearn': r2,
    'KNN Manual': r3,
    'KNN sklearn': r4
}, index=['Accuracy', 'Precision', 'Recall'])

print("\n" + "="*60)
print("COMPARACIÓN DE RESULTADOS")
print("="*60)
print(resultados)
print("="*60)

In [ ]:
# Gráfico comparativo
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

resultados.T['Accuracy'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Accuracy')
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3)

resultados.T['Precision'].plot(kind='bar', ax=axes[1], color='orange')
axes[1].set_title('Precision')
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3)

resultados.T['Recall'].plot(kind='bar', ax=axes[2], color='green')
axes[2].set_title('Recall')
axes[2].set_ylim([0, 1])
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Análisis de Cierre

### a) ¿Cuál implementación fue mejor y por qué?

Comparando los resultados, podemos ver que las implementaciones manuales dan resultados muy similares a las de sklearn, lo cual es bueno porque significa que implementamos correctamente los algoritmos.

Entre Regresión Logística y KNN:
- La **Regresión Logística** crea una frontera de decisión lineal (una línea recta). Es más simple y rápida.
- El **KNN** crea fronteras más complejas y no lineales. Puede capturar patrones más complicados pero es más lento al predecir.

En este caso, [analiza tus resultados y completa]: si ambos tienen métricas similares, es mejor usar Regresión Logística porque es más interpretable y eficiente. Si KNN tiene mejor desempeño, significa que los datos tienen patrones no lineales que la regresión logística no puede capturar.

### b) ¿Qué error es más costoso en detección de Phishing?

En el contexto de phishing, tenemos dos tipos de errores:

1. **Falso Positivo**: Bloquear un sitio legítimo pensando que es phishing
   - Consecuencia: El usuario no puede entrar a un sitio seguro
   - Es molesto pero no peligroso

2. **Falso Negativo**: Dejar pasar un sitio de phishing como si fuera legítimo
   - Consecuencia: El usuario entra al sitio malicioso y le pueden robar sus datos, contraseñas o dinero
   - Esto SÍ es muy peligroso

**Respuesta**: Los **Falsos Negativos** son más costosos porque significan que estamos dejando pasar ataques de phishing. Es preferible bloquear algunos sitios legítimos (y que el usuario los reporte) que dejar pasar sitios maliciosos.

**Métrica a priorizar**: **RECALL**

El Recall nos dice: de todos los sitios de phishing reales, ¿cuántos detectamos? Queremos que este número sea lo más alto posible, incluso si eso significa tener algunos falsos positivos. La fórmula es:

```
Recall = Verdaderos Positivos / (Verdaderos Positivos + Falsos Negativos)
```

Un Recall alto significa pocos falsos negativos, que es exactamente lo que necesitamos en seguridad.